In [2]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1)
print(outputs.keys())
print(outputs.text_embeds.shape)

/home/tiennv/.conda/envs/inpaint/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tiennv/.conda/envs/inpaint/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])
torch.Size([2, 768])


In [3]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k")
processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# print(outputs.logits_per_image)
# probs = logits_per_image.softmax(dim=1)
print(outputs.keys())
print(outputs.text_embeds.shape)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])
torch.Size([2, 1280])


In [4]:
from transformers import AutoTokenizer, PretrainedConfig
def import_model_class_from_model_name_or_path(
    pretrained_model_name_or_path: str, revision: str, subfolder: str = "text_encoder"
):
    text_encoder_config = PretrainedConfig.from_pretrained(
        pretrained_model_name_or_path, subfolder=subfolder, revision=revision
    )
    model_class = text_encoder_config.architectures[0]

    if model_class == "CLIPTextModel":
        from transformers import CLIPTextModel

        return CLIPTextModel
    elif model_class == "CLIPTextModelWithProjection":
        from transformers import CLIPTextModelWithProjection

        return CLIPTextModelWithProjection
    else:
        raise ValueError(f"{model_class} is not supported.")
pretrained_model_name_or_path = "stabilityai/stable-diffusion-xl-base-1.0"
revision = None
variant = None
text_encoder_cls_two = import_model_class_from_model_name_or_path(
    pretrained_model_name_or_path, revision, subfolder="text_encoder_2"
)
text_encoder_two = text_encoder_cls_two.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder_2", revision=revision, variant=variant
)

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [5]:
print(text_encoder_two.get_output_embeddings)

<bound method PreTrainedModel.get_output_embeddings of CLIPTextModelWithProjection(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 1280)
      (position_embedding): Embedding(77, 1280)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-31): 32 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (layer_norm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1280, out_features=5120, bias=True)
            (fc2): Linear(in_features=5120, out_featur

In [6]:
from transformers import AutoImageProcessor, ViTMAEForPreTraining
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-base')
model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
# print(outputs.keys())
print(outputs.logits.shape)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


torch.Size([1, 196, 768])


In [7]:
from transformers import AutoImageProcessor, ViTMAEForPreTraining
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-large')
model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-large')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
# print(outputs.keys())
print(outputs.logits.shape)
# loss = outputs.loss
# mask = outputs.mask
# ids_restore = outputs.ids_restore

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


torch.Size([1, 196, 768])


In [8]:
from transformers import AutoImageProcessor, ViTMAEForPreTraining
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-huge')
model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-huge')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
print(outputs.logits.shape)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


torch.Size([1, 256, 588])


In [14]:
tokenizer_two = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    subfolder="tokenizer_2",
    revision=revision,
    use_fast=False,
)
text_encoder_cls_one = import_model_class_from_model_name_or_path(
    pretrained_model_name_or_path, revision
)
text_encoder_one = text_encoder_cls_one.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder_2", revision=revision, variant=variant
)
print(tokenizer_two.model_max_length)
test_text = "this is a test text for the tokenizer" * 100
test_input = [test_text, test_text]
text_inputs = tokenizer_two(test_input, return_tensors="pt", padding="max_length", max_length=tokenizer_two.model_max_length, truncation=True)
# print(inputs['input_ids'])
text_input_ids = text_inputs.input_ids
prompt_embeds = text_encoder_one(
    text_input_ids.to(text_encoder_one.device),
    output_hidden_states=True,
)
print(prompt_embeds.keys())
pooled_prompt_embeds = prompt_embeds[0]
prompt_embeds = prompt_embeds[-1][-2]
bs_embed, seq_len, _ = prompt_embeds.shape
prompt_embeds = prompt_embeds.view(bs_embed, seq_len, -1)
print(f"bs_embeds: {bs_embed}, seq_len: {seq_len}")
print(f"prompt_embeds: {prompt_embeds.shape}")
print(f"pooled_prompt_embeds: {pooled_prompt_embeds.shape}")
prompt_embeds = prompt_embeds.view(bs_embed, seq_len, -1)
print(f"prompt_embeds: {prompt_embeds.shape}")
# prompt_embeds[0].shape
pooled_prompt_embeds = pooled_prompt_embeds.view(bs_embed, -1)
print(f"pooled_prompt_embeds: {pooled_prompt_embeds.shape}")

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


77
odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])
bs_embeds: 2, seq_len: 77
prompt_embeds: torch.Size([2, 77, 1280])
pooled_prompt_embeds: torch.Size([2, 77, 1280])
prompt_embeds: torch.Size([2, 77, 1280])
pooled_prompt_embeds: torch.Size([2, 98560])
